#  爬蟲-MITRE ATT&CK®
---------------
這部分是抓取MITRE ATT&CK®這個網站中的惡意軟體攻擊戰略的每個技術，用於製作輸入給ChatGPT的提示。

MITRE ATT&CK®這個網站是全球開放地資訊攻防戰略資料庫，且所有技術與戰略街是從真實世界的案例中研究並且記錄下來。
因此在專題中我們使用這個網站作為戰術的資料來源。

白話文:
　這個網站記錄了資安打架的技巧，所以它的資料可以作為我們訓練GPT如何打架的資寮來源。
#
--------------

### 我們所有的研究都是基於研究資料安全，以「知己知彼，百戰百勝」的精神在進行對惡意軟體以及攻擊來進一步規劃如何防禦與提升安全意識的研究。

In [3]:
from bs4 import BeautifulSoup
import requests

url='https://attack.mitre.org/matrices/enterprise/'

web = requests.get(url)                        # 取得網頁內容
soup = BeautifulSoup(web.text, "html.parser")  # 轉換成標籤樹
td=soup.select('td')
dic_MitreAttack={}
for i in td:
    if i.get("class") == ['tactic', 'name']:
        print(i.a["href"])
        print(i.a.text)
        dic_MitreAttack[i.a.text]={}
        url_i="https://attack.mitre.org"+i.a["href"]
        dic_MitreAttack[i.a.text]=tactics_to_unique(url_i)

/tactics/TA0043
Reconnaissance
https://attack.mitre.org/tactics/TA0043 <Response [200]>
/tactics/TA0042
Resource Development
https://attack.mitre.org/tactics/TA0042 <Response [200]>
/tactics/TA0001
Initial Access
https://attack.mitre.org/tactics/TA0001 <Response [200]>
/tactics/TA0002
Execution
https://attack.mitre.org/tactics/TA0002 <Response [200]>
/tactics/TA0003
Persistence
https://attack.mitre.org/tactics/TA0003 <Response [200]>
/tactics/TA0004
Privilege Escalation
https://attack.mitre.org/tactics/TA0004 <Response [200]>
/tactics/TA0005
Defense Evasion
https://attack.mitre.org/tactics/TA0005 <Response [200]>
/tactics/TA0006
Credential Access
https://attack.mitre.org/tactics/TA0006 <Response [200]>
/tactics/TA0007
Discovery
https://attack.mitre.org/tactics/TA0007 <Response [200]>
/tactics/TA0008
Lateral Movement
https://attack.mitre.org/tactics/TA0008 <Response [200]>
/tactics/TA0009
Collection
https://attack.mitre.org/tactics/TA0009 <Response [200]>
/tactics/TA0011
Command and Con

### 將個別TTPs依據Tactics分類儲存至dictionary中

In [4]:
def tactics_to_unique(url_test):
    ID="error"
    web_test = requests.get(url_test)
    print(url_test,web_test)
    soup_test = BeautifulSoup(web_test.text, "html.parser")
    tr_test=soup_test.select('tr')
    #print(tr_test)
    dic_tc={}
    #輪流檢視每個Technique
    #View each technique in turn
    for i in tr_test:
        tactics_list_test=i.select('td')
        list_len=[]
        list_technique=[]
        #把該位置的ID,Name,Description都放進list
        #append ID,Name Description into a list
        for j in tactics_list_test:
            description=j.text.splitlines()
        
            #print(description)
            #if len(description)>1:print(description[0])
            list_len.append(len(description))
            
        #判斷這是technique還是sub technique
        #tell if it's of technique or of sub technique
        tag_technique=0
        if list_len==[0,2,2,4]:
            #print("Yes,sub technique")
            count_catch_technique=0
            tag_technique=2
            for j in tactics_list_test:
                #print(j)
                count_catch_technique+=1
                if count_catch_technique >1:
                    list_technique.append(j.text)
        elif list_len==[2,2,4]:
            #print("Yes,technique")
            count_catch_technique=0
            tag_technique=1
            for j in tactics_list_test:
                count_catch_technique+=1
                if count_catch_technique !=0:
                        list_technique.append(j.text)
    #print(list_technique)
        list_technique_=[]
        count_dic=0
        
        for k in list_technique:
            #print(k.splitlines()[1]) 
            count_dic+=1
            for l in k.splitlines()[1].split("                                "):
                #print(l)
                #print(count_dic)            

                
                if tag_technique==1:
                    if count_dic==1:
                        dic_tc[l]={}
                        ID=l
                        dic_tc[ID]["Sub Technique"]={}
                        dic_tc[ID]["ID"]=ID.split(" ")[1]
                    elif count_dic==2:
                        dic_tc[ID]["Name"]=l
                    elif count_dic==3:
                        dic_tc[ID]["Description"]=l                    
                else:
                    if count_dic==1:
                        dic_tc[ID]["Sub Technique"][l]={}
                        dic_tc[ID]["Sub Technique"][l]["ID"]=l
                        subtechniqueID=l
                    elif count_dic==2:
                        dic_tc[ID]["Sub Technique"][subtechniqueID]["Name"]=l
                    elif count_dic==3:
                        dic_tc[ID]["Sub Technique"][subtechniqueID]["Description"]=l
    return dic_tc
url_test="https://attack.mitre.org/tactics/TA0003/"
tactics_to_unique(url_test)  

https://attack.mitre.org/tactics/TA0003/ <Response [200]>


{' T1098 ': {'Sub Technique': {' .001 ': {'ID': ' .001 ',
    'Name': ' Additional Cloud Credentials ',
    'Description': '    Adversaries may add adversary-controlled credentials to a cloud account to maintain persistent access to victim accounts and instances within the environment.'},
   ' .002 ': {'ID': ' .002 ',
    'Name': ' Additional Email Delegate Permissions ',
    'Description': '    Adversaries may grant additional permission levels to maintain persistent access to an adversary-controlled email account. '},
   ' .003 ': {'ID': ' .003 ',
    'Name': ' Additional Cloud Roles ',
    'Description': '    An adversary may add additional roles or permissions to an adversary-controlled cloud account to maintain persistent access to a tenant. For example, adversaries may update IAM policies in cloud-based environments or add a new global administrator in Office 365 environments. With sufficient permissions, a compromised account can gain almost unlimited access to data and settings

### 多製作一份不分類只有儲存TTPs的dictionary

In [5]:
dic_TTPs={}
for i in dic_MitreAttack:
    #print(dic_MitreAttack[i])
    for j in dic_MitreAttack[i]:
        #print(dic_MitreAttack[i][j])
        dic_TTPs[j]=dic_MitreAttack[i][j]
#dic_TTPs

## 儲存結果

In [6]:
import json
with open('dic_MitreAttack.json',"w+") as file_json:
    json.dump(dic_MitreAttack,file_json,indent=2)
with open('dic_TTPs.json',"w+") as file_json:
    json.dump(dic_TTPs,file_json,indent=2)